# Frozen Bot
Heladerías Frozen SRL nos solicita la construcción de un bot para la toma de
pedidos. Es nuestra responsabilidad el desarrollo de funciones auxiliares que le darán al bot la capacidad de desenvolverse en la conversación.